In [14]:
!pip install jsonpath_ng

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
import os
import json
import numpy as np
import pandas as pd
import sqlite3
import functools as ft
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from jsonpath_ng import parse
from enum import Enum
%matplotlib inline

# Source to Target Mapping - OOP Approach for Single Table

### Data

In [16]:

import json
with open("Glassdoor Gender.json", "r") as file:
    data = json.load(file)
    for item in data:
        item["JobTitle"] = str(item["JobTitle"])
data[:10]

[{'JobTitle': 'Graphic Designer',
  'Gender': 'Female',
  'Age': 18,
  'PerfEval': 5,
  'Education': 'College',
  'Dept': 'Operations',
  'Seniority': 2,
  'Salary': 42363,
  'Bonus': 9938,
  'Require_level': 'Junior',
  'ID_employee': 6789},
 {'JobTitle': 'Warehouse Associate',
  'Gender': 'Female',
  'Age': 19,
  'PerfEval': 4,
  'Education': 'PhD',
  'Dept': 'Administration',
  'Seniority': 5,
  'Salary': 90208,
  'Bonus': 9268,
  'Require_level': 'medium',
  'ID_employee': 6790},
 {'JobTitle': 'IT',
  'Gender': 'Female',
  'Age': 20,
  'PerfEval': 5,
  'Education': 'PhD',
  'Dept': 'Operations',
  'Seniority': 4,
  'Salary': 70890,
  'Bonus': 10126,
  'Require_level': 'medium',
  'ID_employee': 6791},
 {'JobTitle': 'Graphic Designer',
  'Gender': 'Female',
  'Age': 20,
  'PerfEval': 5,
  'Education': 'College',
  'Dept': 'Sales',
  'Seniority': 4,
  'Salary': 67585,
  'Bonus': 10541,
  'Require_level': 'medium',
  'ID_employee': 6792},
 {'JobTitle': 'Graphic Designer',
  'Gender': 

###  Abstract Base Calss (ABC) for mutual methods

In [17]:
class Interface(ABC):

    @abstractmethod
    def get_data_by_field(self, field_name):
        """Fetch the data by given feild name """

    @abstractmethod
    def get_data_by_id(self, id):
        """Fetch the data by given ID  """

    @abstractmethod
    def get(self):
        """Fetch all data """

### Transform Operations
inherithed from Enum - class that automatic enumrate the variables

In [38]:
class TransformMask(Enum):
    # add here any masks you want
    CLEAN_STRING = ".strip().lower()"
    CAPITAL_LETTER = ".strip().lower().title()"
    CLEAN_NUM = ""


### Database Class - Define Common Properties for Source, Target, Mapping

In [37]:
class Database:
    def __init__(self):
        self.db = {
            "source": [],
            "destination": [],
            "transform": [],
            "mapping": []
        }
        self.add_source(1, "JobTitle", "str")
        self.add_destination("Fact table-Salary",1, "JobTitle", "str")
        self.add_transform(1, "CLEAN_STRING")
        self.add_mapping("Fact table-Salary", 1, 1, 1,1)

        self.add_source(2, "Salary", "int")
        self.add_destination("Fact_table_Salary",2, "Salary", "float")
        self.add_transform(2, "CLEAN_NUM")
        self.add_mapping("Fact table-Salary", 2, 2, 2,2)

        self.add_source(3, "Bonus", "int")
        self.add_destination("Fact table-Salary",3, "Bonus", "float")
        self.add_transform(3, "CLEAN_NUM")
        self.add_mapping("Fact table-Salary", 3, 3, 3,3)

        self.add_source(4, "ID_employee", "int")
        self.add_destination("Fact table-Salary",4, "ID_employee", "int")
        self.add_transform(4, "CLEAN_NUM")
        self.add_mapping("Fact table-Salary", 4, 4, 4,4)

        self.add_source(5, "Gender", "str")
        self.add_destination("Dimention_table_Emploee",5, "JobTitle", "str")
        self.add_transform(5, "CAPITAL_LETTER")
        self.add_mapping("Dimention_table_Emploee", 5, 5, 5,5)

        self.add_source(6, "Age", "int")
        self.add_destination("Fact_table_Salary",6, "Salary", "int")
        self.add_transform(6, "CLEAN_NUM")
        self.add_mapping("Fact table-Salary", 6, 6, 6,6)

        self.add_source(7, "PerfEval", "int")
        self.add_destination("Dimention_table_Emploee",7, "PerfEval", "int")
        self.add_transform(7, "CLEAN_NUM")
        self.add_mapping("Dimention_table_Emploee", 7, 7, 7,7)

        self.add_source(8, "Education", "str")
        self.add_destination("Dimention_table_Emploee",8, "Education", "str")
        self.add_transform(8, "CAPITAL_LETTER")
        self.add_mapping("Dimention_table_Emploee", 8, 8, 8,8)

        self.add_source(9, "Dept", "str")
        self.add_destination("Dimention_table_job",9, "Dept", "str")
        self.add_transform(9, "CAPITAL_LETTER")
        self.add_mapping("Dimention_table_job", 9, 9, 9,9)

        self.add_source(10, "Require_level", "str")
        self.add_destination("Dimention_table_job",10, "Require_level", "str")
        self.add_transform(9, "CAPITAL_LETTER")
        self.add_mapping("Dimention_table_job", 10, 10, 10,10)

        self.add_source(11, "Seniority", "int")
        self.add_destination("Dimention_table_Emploee",11, "Seniority", "int")
        self.add_transform(11, "CLEAN_NUM")
        self.add_mapping("Dimention_table_Emploee", 11, 11, 11,11)



    def add_source(self, id, field, typ):
            self.db["source"].append({
                "id": id,
                "source_field_name": field,
                "source_field_mapping": field,
                "source_field_type": typ,
                "is_required": True,
            })

    def add_destination(self,table, id, field, typ):
        self.db["destination"].append({
            "id": id,
            "destination_field_name": field,
            "destination_field_mapping": field,
            "destination_field_type": typ,
            "default_value": "n/a",
            "destination_table":table
        })

    def add_transform(self, id, mask):
        self.db["transform"].append({
            "id": id,
            "transform_mask": mask,
        })

    def add_mapping(self,table ,id, source, dest, transform):
      self.db["mapping"].append({
          "id": id,
          "mapping_source": source,
          "mapping_destination": dest,
          "mapping_transform": transform,
          "destination_table":table
        })


    @property
    def get_data_source_target_mapping(self):
      return self.db


### Source class

Inherited from Interface for the common methods and from Database for common variables

In [39]:
class Source(Interface, Database):
    def __init__(self):
        Database.__init__(self)

    # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("source")

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id") == self.id:
                return x
        return None

### Target class

Inherited from Interface for the common methods and from Database for common variables

In [40]:
class Target(Interface, Database):

    def __init__(self):
        Database.__init__(self)

    # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("destination")

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

### Transform Class

Inherited from Interface for the common methods and from Database for common variables

In [41]:
class Transform(Interface, Database):

    def __init__(self):
        Database.__init__(self)

    # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("transform", [])

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

### Mapping class

Inherited from Interface for the common methods and from Database for common variables

In [42]:
class Mappings(Interface, Database):

    def __init__(self):
        Database.__init__(self)

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("mapping")

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

    def get_data_by_field(self, field_name):
        return None

### Format Class - JSON

Search the source data value inside a JSON file

In [43]:
class JsonQuery:
    def __init__(self, json_path, json_data):
        self.json_path = json_path
        self.json_data = json_data

    def get(self):
        jsonpath_expression = parse(self.json_path)
        match = jsonpath_expression.find(self.json_data)
        source_data_value = match[0].value
        return source_data_value

### Combine it All - STTM

In [44]:
class STTM:
    def __init__(self, input_json):
        self.input_json = input_json
        self.mapping_instance = Mappings()
        self.source_instance = Source()
        self.destination_instance = Target()
        self.transform_instance = Transform()
        self.look_up_mask = {i.name: i.value for i in TransformMask}
        self.json_data_transformed = {}
        self.to_table = {}

    def _get_mapping_data(self):
        return self.mapping_instance.get

    def _get_mapping_source_data(self):
        return self.source_instance.get

    def get_transformed_data(self):


        for mappings in self._get_mapping_data():

            """fetch the source mapping """
            mapping_source_id = mappings.get("mapping_source")
            mapping_destination_id = mappings.get("mapping_destination")
            mapping_transform_id = mappings.get("mapping_transform")
            mapping_table = mappings.get("destination_table")

            mapping_source_data = self.source_instance.get_data_by_id(id=mapping_source_id)
            transform_data = self.transform_instance.get_data_by_id(id=mapping_transform_id)

            """Fetch Source  field Name"""
            source_field_name = mapping_source_data.get("source_field_name")

            """if field given is not present incoming json """
            if source_field_name not in self.input_json.keys():
                if mapping_source_data.get("is_required"):
                    raise Exception(
                        "Alert ! Field {} is not present in JSON please FIX mappings ".format(source_field_name))
                else:
                    pass

            else:
                source_data_value = JsonQuery(
                    json_path=mapping_source_data.get("source_field_mapping"),
                    json_data=self.input_json
                ).get()

                """check the data type for source if matches with what we have """
                if mapping_source_data.get("source_field_type") != type(source_data_value).__name__:
                    if source_data_value is not None:
                        _message = (
                            "Alert ! Source Field :{} Datatype has changed from {} to {} ".format(source_field_name,
                                                                                                  mapping_source_data.get(
                                                                                                      "source_field_type"),
                                                                                                  type(
                                                                                                      source_data_value).__name__))
                        print(_message)
                        raise Exception(_message)

                """Query and fetch the Destination | target """
                destination_mappings_json_object = self.destination_instance.get_data_by_id(
                    id=mappings.get("mapping_destination"))

                destination_field_name = destination_mappings_json_object.get("destination_field_name")
                destination_field_type = destination_mappings_json_object.get("destination_field_type")
                #destination_table = destination_mappings_json_object.get("destination_table")
                self.to_table[destination_field_name]=mapping_table
                #temp[destination_field_name]=mapping_table

                dtypes = [str, float, list, int, set, dict]

                for dtype in dtypes:

                    """Datatype Conversion """
                    if destination_field_type == str(dtype.__name__):

                        """is source is none insert default value"""
                        if source_data_value is None:
                            self.json_data_transformed[destination_field_name] = dtype.__call__(
                                destination_mappings_json_object.get("default_value")
                            )

                        else:
                            """check if you have items to transform"""
                            if transform_data is not None:
                                """ check for invalid mask name """
                                if transform_data.get("transform_mask") not in list(self.look_up_mask.keys()):
                                    raise Exception(
                                        f"Specified Transform {transform_data.get('transform_mask')} is not available please select from following Options :{list(self.look_up_mask.keys())}")
                                else:
                                    mask_apply = self.look_up_mask.get(transform_data.get("transform_mask"))
                                    converted_dtype = dtype.__call__(source_data_value)
                                    mask = f'converted_dtype{mask_apply}'
                                    curated_value = eval(mask)
                                    self.json_data_transformed[destination_field_name] = curated_value

                            else:
                                self.json_data_transformed[destination_field_name] = dtype.__call__(source_data_value)

        return self.json_data_transformed,self.to_table

In [45]:
transformed_data = []
for item in data:
    helper = STTM(input_json=item)
    response,mapping = helper.get_transformed_data()
    transformed_data.append(response)
    print(response)
print(mapping)

{'JobTitle': 'Female', 'Salary': 18, 'Bonus': 9938.0, 'ID_employee': 6789, 'PerfEval': 5, 'Education': 'College', 'Dept': 'Operations', 'Require_level': 'Junior', 'Seniority': 2}
{'JobTitle': 'Female', 'Salary': 19, 'Bonus': 9268.0, 'ID_employee': 6790, 'PerfEval': 4, 'Education': 'Phd', 'Dept': 'Administration', 'Require_level': 'medium', 'Seniority': 5}
{'JobTitle': 'Female', 'Salary': 20, 'Bonus': 10126.0, 'ID_employee': 6791, 'PerfEval': 5, 'Education': 'Phd', 'Dept': 'Operations', 'Require_level': 'medium', 'Seniority': 4}
{'JobTitle': 'Female', 'Salary': 20, 'Bonus': 10541.0, 'ID_employee': 6792, 'PerfEval': 5, 'Education': 'College', 'Dept': 'Sales', 'Require_level': 'medium', 'Seniority': 4}
{'JobTitle': 'Female', 'Salary': 33, 'Bonus': 9836.0, 'ID_employee': 6793, 'PerfEval': 5, 'Education': 'High School', 'Dept': 'Engineering', 'Require_level': 'medium', 'Seniority': 5}
{'JobTitle': 'Female', 'Salary': 35, 'Bonus': 9941.0, 'ID_employee': 6794, 'PerfEval': 5, 'Education': 'Col

In [46]:
df=pd.DataFrame(transformed_data)
df

,JobTitle,Salary,Bonus,ID_employee,PerfEval,Education,Dept,Require_level,Seniority
0,Female,18,9938.0,6789,5,College,Operations,Junior,2
1,Female,19,9268.0,6790,4,Phd,Administration,medium,5
2,Female,20,10126.0,6791,5,Phd,Operations,medium,4
3,Female,20,10541.0,6792,5,College,Sales,medium,4
4,Female,33,9836.0,6793,5,High School,Engineering,medium,5
...,...,...,...,...,...,...,...,...,...
995,Male,62,1823.0,7784,1,Masters,Administration,Junior,2
996,Male,57,3567.0,7785,1,Masters,Sales,Junior,2
997,Male,48,2724.0,7786,1,High School,Operations,Junior,1
998,Male,65,2225.0,7787,2,High School,Administration,Junior,1
